# Protocols
https://docs.netsquid.org/latest-release/tutorial.protocols.html

In [2]:
import netsquid as ns
from netsquid.protocols import Protocol

class WaitProtocol(Protocol):
    def run(self):
        print(f'starting protocol as {ns.sim_time()}')
        yield self.await_timer(100)
        print(f'ending protocol at {ns.sim_time()}')

ns.sim_reset()
protocol = WaitProtocol()
protocol.start()

starting protocol as 0.0


WaitProtocol('WaitProtocol')

In [3]:
stats = ns.sim_run()

ending protocol at 100.0


## The ping pong example using protocols

In [4]:
from netsquid.protocols import NodeProtocol
from netsquid.components import QuantumChannel
from netsquid.nodes import Node, DirectConnection
from netsquid.qubits import qubitapi as qapi

In [5]:
class PingProtocol(NodeProtocol):
    def run(self):
        print(f'Starting ping at t={ns.sim_time()}')
        port = self.node.ports['port_to_channel']
        qubit, = qapi.create_qubits(1)
        port.tx_output(qubit)   # send qubit to pong
        while True:
            # wait for qubit to be received back
            yield self.await_port_input(port)
            qubit = port.rx_input().items[0]
            m, prob = qapi.measure(qubit, ns.Z)
            labels_z = ('|0>', '|1>')
            print(f'{ns.sim_time()}: pong event! {self.node.name} measured {labels_z[m]} with probability {prob:.2f}')
            port.tx_output(qubit)   # send qubit to B

In [6]:
class PongProtocol(NodeProtocol):
    def run(self):
        print('starting pong at t={}'.format(ns.sim_time()))
        port = self.node.ports['port_to_channel']
        while True:
            yield self.await_port_input(port)
            qubit = port.rx_input().items[0]
            m, prob = qapi.measure(qubit, ns.X)
            labels_x = ('|+>', '|->')
            print(f'{ns.sim_time()}: ping event! {self.node.name} measured {labels_x[m]} with probability {prob:.2f}')
            port.tx_output(qubit)

In [18]:
ns.sim_reset()
ns.set_random_state(seed=42)
node_ping = Node('Ping', port_names=['port_to_channel'])
node_pong = Node('Pong', port_names=['port_to_channel'])
connection = DirectConnection('Connection', QuantumChannel('Channel_LR', delay=10), QuantumChannel('Channel_RL', delay=10))
node_ping.ports['port_to_channel'].connect(connection.ports['A'])
node_pong.ports['port_to_channel'].connect(connection.ports['B'])
ping_protocol = PingProtocol(node_ping)
pong_protocol = PongProtocol(node_pong)

In [19]:
ping_protocol.start()

Starting ping at t=0.0


PingProtocol(Node(name='Ping'), 'PingProtocol')

In [20]:
pong_protocol.start()

starting pong at t=0.0


PongProtocol(Node(name='Pong'), 'PongProtocol')

In [21]:
stats = ns.sim_run(91)

10.0: ping event! Pong measured |+> with probability 0.50
20.0: pong event! Ping measured |1> with probability 0.50
30.0: ping event! Pong measured |-> with probability 0.50
40.0: pong event! Ping measured |1> with probability 0.50
50.0: ping event! Pong measured |+> with probability 0.50
60.0: pong event! Ping measured |0> with probability 0.50
70.0: ping event! Pong measured |+> with probability 0.50
80.0: pong event! Ping measured |1> with probability 0.50
90.0: ping event! Pong measured |-> with probability 0.50


In [22]:
print(stats)


Simulation summary

Elapsed wallclock time: 0:00:00.001827
Elapsed simulation time: 9.10e+01 [ns]
Triggered events: 18
Handled callbacks: 18
Total quantum operations: 9
Frequent quantum operations: MEASURE = 9
Max qstate size: 1 qubits
Mean qstate size: 1.00 qubits



In [23]:
pong_protocol.stop()
stats = ns.sim_run()

100.0: pong event! Ping measured |1> with probability 0.50


In [24]:
pong_protocol.start()

starting pong at t=110.0


PongProtocol(Node(name='Pong'), 'PongProtocol')

In [25]:
ping_protocol.reset()

Starting ping at t=110.0


In [26]:
stats = ns.sim_run(duration=51)

120.0: ping event! Pong measured |+> with probability 0.50
130.0: pong event! Ping measured |1> with probability 0.50
140.0: ping event! Pong measured |-> with probability 0.50
150.0: pong event! Ping measured |0> with probability 0.50
160.0: ping event! Pong measured |+> with probability 0.50
